<a href="https://colab.research.google.com/github/mahinuralam/notebook/blob/master/Proposed_Model_MES_Prediction_Multi_MLP_LightGBM_FS_15_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Input, BatchNormalization, ReLU, Bidirectional, GRU, LeakyReLU, Activation, Concatenate
from keras.models import Model
import tensorflow as tf


from google.colab import drive
drive.mount('/content/drive')


predict_train_data = pd.read_csv('/content/drive/MyDrive/Google Colab/MES Project/Dataset/MES_Prediction_Model/FS_Implemented/ML-based FS/LightGBM-FS/LightGBM_Dataset/Kaggle_Training_Dataset_v2_LightGBM_FS_15_Features.csv')
predict_test_data = pd.read_csv('/content/drive/MyDrive/Google Colab/MES Project/Dataset/MES_Prediction_Model/FS_Implemented/ML-based FS/LightGBM-FS/LightGBM_Dataset/Kaggle_Testing_Dataset_v2_LightGBM_FS_15_Features.csv')


predict_train_data = predict_train_data.drop('Unnamed: 0',axis=1)
predict_test_data = predict_test_data.drop('Unnamed: 0',axis=1)

# Important features selection
features = []
for feature in predict_train_data.columns:
    if feature != 'went_on_backorder':
        features.append(feature)
X_train = predict_train_data[features]
y_train = predict_train_data['went_on_backorder']

# Dataset spliting
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_test_res, y_test_res = sm.fit_resample(X_test, y_test)

# Data preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_res = scaler.fit_transform(X_train_res)
X_test_res = scaler.fit_transform(X_test_res)

# Model traning parameters
import tensorflow as tf
import keras
my_metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

# MLP Model
def Multi_MLP():
    visible = Input(shape=(15,))
    hidden_1a = Dense(100, activation='relu')(visible)
    hidden_2a = Dense(50, activation='relu')(hidden_1a)
    hidden_3a = Dense(25, activation='relu')(hidden_2a)
    hidden_1b = Dense(100, activation='relu')(visible)
    hidden_2b = Dense(50, activation='relu')(hidden_1b)
    hidden_3b = Dense(25, activation='relu')(hidden_2b)
    concat = Concatenate()([hidden_3a, hidden_3b])
    output = Dense(1, activation='sigmoid')(concat)
    model = Model(inputs=visible, outputs=output)
    return model


# Build Model
model = Multi_MLP()
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# Model Traning
model.compile(loss='mean_squared_error', optimizer=opt, metrics=my_metrics)
model_history = model.fit(X_train_res, y_train_res, batch_size=32, epochs=50, verbose=2)


# Generate predictions
trainpred = model.predict(X_train_res)
valpred = model.predict(X_test_res)

# Output visualization
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 26})
epochs = range(1, len(valpred) + 1)
plt.figure(figsize=(20, 5))

plt.plot(epochs, valpred, 'b', label='Predict')
plt.plot(epochs, y_test_res, 'g', label='Actual')
plt.axis([0, 10000, 0, 1])
plt.xlabel('Sampling index')
plt.ylabel('Re-ordered status')
plt.legend(loc='upper left')
plt.savefig('predict_lightgbm_v3.png')

